In [ ]:
% 用以形成图2的matlab代码
​
load seismo_w.mat;
dt = 0.00025;
offset = 5;
dx = 2;
vmin = 100;
dv = 1;
vmax = 600;
fmin = 5;
fmax = 60;
​
[E,freq,v] = PhaseShiftOfSW(seismo_w,dt,offset,dx,vmin,dv,vmax,fmin,fmax);
​
x=[freq(1) freq(end)];
y=[v(1) v(end)];
​
figure(1)
imagesc(x,y,E);
colormap(jet);
colorbar;
set(gca,'YDir','normal');
xlabel('频率/(Hz)');
ylabel('相速度/(m/s)');

In [ ]:
try:
    print()
except Exception as e:
    print(f'error, {i}')
    continue

In [ ]:
function [E,freq,v] = PhaseShiftOfSW(rec,dt,offset,dx,vmin,dv,vmax,fmin,fmax)
%   Summary of this function goes here.
%   [E,freq,v] = PhaseShiftOfSW(rec,dt,offset,dx,vmin,dv,vmax,fmin,fmax)
%   Detailed explanation goes here.
%   The function is for generating the dispersive energy of surface waves.
%   The function is generally adopted in Multi-channel Analysis of Surface Waves (MASW).
%
%   IN      
%           rec: the seismic record of surface waves, the size of rec is [nt*nx].
%            dt: the sampling interval in time domain (s).
%        offset: the distance (m) of the source to the first receiver.
%            dx: the spacing (m) of the receiver.
%          vmin: the minimum scanned phase velocity (m/s).
%            dv: the interval of the scanned phase velocity (m/s). 
%          vmax: the maximum scanned phase velocity (m/s).
%          fmin: the minimum frequency (Hz) of the dispersive energy.
%          fmax: the maximum frequency (Hz) of the dispersive energy.
%
%  OUT   
%             E: the normalized dispersive energy.
%          freq: the frequency (Hz) vecotor of the normalized dispersive energy.
%             v: the phase velocity (m/s) vector of the normalized dispersive energy.
%  
%   References: 
%           Park CB, Miller RD, Xia J. Imaging dispersion curves of surface waves 
%           on multi-channel record[C]. Technical Program with Biographies SEG, 
%           68th Annual Meeting, New Orleans, L A., 1998, 1377-1380.
%
%  Author(s): Yan Yingwei
%  Copyright: 2020-2025 
%  Revision: 1.0  Date: 5/28/2020
%
%  Academy of Opto-Electronics, China Electronic Technology Group Corporation (AOE CETC)
​
​
​
[nt,nx] = size(rec);      % 获得地震记录的维度
rec_fft = zeros(nt,nx);   % 地震记录的频谱
​
v=vmax:-dv:vmin;          % 相速度扫描向量
lv = length(v);           % 相速度向量的长度
​
Fs = 1/dt;                % 采样率
f = Fs*(0:(nt/2))/nt;     % 频率向量
​
% 确定频散能量图中最小频率在频率向量中的索引
ind = find(f>fmin);       
fmin_ind  = ind(1)-1;
​
% 确定频散能量图中最大频率在频率向量中的索引
ind = find(f>fmax);
fmax_ind = ind(1);
​
freq = f(fmin_ind:fmax_ind); % 频散能量图中的频率向量
lf = length(freq);
​
for j=1:nx
    rec_fft(:,j) = fft(rec(:,j),nt,1); % 获得地震记录的频谱
end
​
rec_fft_amp = abs(rec_fft);
​
rec_fft_n = rec_fft./rec_fft_amp;      % 对地震记录的频谱作归一化
​
E = zeros(lv,lf); % 频散能量矩阵
​
​
for j=1:lf
    f_ind = fmin_ind+j-1;
    for i=1:lv
        for k=1:nx
            x=offset+(k-1)*dx;
            E(i,j) = E(i,j)+exp(1i*2*pi*f(f_ind)*x/v(i))*rec_fft_n(f_ind,k);
        end
    end
    E(:,j) = abs(E(:,j)./max(abs(E(:,j))));  % 对每个频率的频散能量作归一化
end
end

In [ ]:
import numpy as np
from scipy.fftpack import fft

def phase_shift_of_sw(rec, dt, offset, dx, vmin, dv, vmax, fmin, fmax):
    nt, nx = rec.shape                              # 获取地震记录的维度
    rec_fft = np.zeros((nt, nx), dtype=complex)     # 地震记录的频谱

    v = np.arange(vmax, vmin - dv, -dv)             # 相速度扫描向量
    lv = len(v)                                     # 相速度向量的长度

    Fs = 1 / dt                                     # 采样率
    f = Fs * np.arange(nt//2 + 1) / nt              # 频率向量

    # 确定频散能量图中最小频率在频率向量中的索引
    fmin_ind = np.where(f > fmin)[0][0] - 1

    # 确定频散能量图中最大频率在频率向量中的索引
    fmax_ind = np.where(f > fmax)[0][0]

    freq = f[fmin_ind:fmax_ind + 1]                 # 频散能量图中的频率向量
    lf = len(freq)

    rec_fft = np.array([fft(rec[:, j]) for j in range(nx)])  # 获得地震记录的频谱
    rec_fft_amp = np.abs(rec_fft)

    rec_fft_n = rec_fft / rec_fft_amp               # 对地震记录的频谱作归一化

    E = np.zeros((lv, lf), dtype=complex)           # 频散能量矩阵

    for j in range(lf):
        f_ind = fmin_ind + j
        for i in range(lv):
            for k in range(nx):
                x = offset + (k) * dx
                E[i, j] += np.exp(1j * 2 * np.pi * f[f_ind] * x / v[i]) * rec_fft_n[f_ind, k]

    for j in range(lf):
        E[:, j] = np.abs(E[:, j] / np.max(np.abs(E[:, j])))  # 对每个频率的频散能量作归一化

    return E, freq, v
